# Import Libraries

In [1]:
import requests # allows for interacting with websites to grab data
import pandas as pd 
import numpy as np

import warnings # these lines remove 'A value is trying to be set on a copy of a slice from a DataFrame.' warning 
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import ipywidgets as widgets # interaction with chart
from ipywidgets import interact, interactive, fixed, interact_manual

import seaborn as sns # style

from datetime import date, datetime, timedelta
pd.set_option('display.max_rows', 200)

# Load Functions

In [2]:
from seas_function import iso_country, wide_to_long, seas_prep, seas_exec, working_days, calendar_adjust, disp_results

# Settings

In [3]:
website = "https://www.acea.be/uploads/statistic_documents/1990%E2%80%932019_PC_by_country_EU%2BEFTA.xlsx"
date_col = 'date'
val_col = 'volume'
val_ca= 'volume_ca'
cat_col = 'country'
range_set = 6 # gives the number of years used to determime seasonality

sns.set_theme()

# wrapper function

In [4]:
df=calendar_adjust(
    working_days(
    iso_country(
    wide_to_long(
    pd.concat(pd.read_excel(requests.get(website).content, sheet_name=None, 
                            skiprows=1, header=1), ignore_index=True).drop(columns = 'FY', index=1),
    '00', date_col, val_col).
        rename(columns=str.lower),
        cat_col),
        date_col), cat_col, val_ca, val_col)

# returns datafram with ACEA data and ISO naming conventions

In [5]:
df_sa=seas_exec(seas_prep(df,
             date_col, 
             cat_col, 
             val_ca), date_col, range_set, cat_col, val_ca)

In [6]:
df=pd.merge(df, 
            df_sa[{'date', 'country','seasonal', 'volume_sa'}], 
            left_on = ['date', 'country'], 
            right_on = ['date', 'country'])

df['csf']=df.calendar_factor*df.seasonal

# Display Results

In [10]:
def disp_results(c):
    
    import matplotlib.pyplot as plt # display results

    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(12,12))
    
    x=df['date'].loc[df.country==c]
    y=df['volume'].loc[df.country==c]
    y2=df['volume_sa'].loc[df.country==c]
    y3=df['seasonal'].loc[df.country==c]
    y4=df['calendar_factor'].loc[df.country==c]
    y5=df.csf.loc[df.country==c]
    
    ax1.set_title('Raw Series vs. Adjusted Series')
    ax2.set_title('Combined Factor')
    ax3.set_title('seasonal factor and calendar factor')
    
    ax1.plot(x, y,
             x, y2)
    ax2.plot(x, y5)
    ax3.plot(x, y3,
             x, y4)

In [11]:
interact(disp_results, c=sorted(df_sa.country.unique()))

interactive(children=(Dropdown(description='c', options=('Austria', 'Belgium', 'Bulgaria', 'Czech Republic', '…

<function __main__.disp_results(c)>

# Output Results

In [ ]:
df.to_csv('df.csv')